# Hyperparamter tuning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import keras_tuner as kt

import numpy as np
import pandas as pd
from IPython import display
from matplotlib import pyplot as plt

import os

print(tf.__version__)

2.18.0


## Importación y pretratamiento de datos
En esta demostración se va a utilizar de nuevo la base de datos de la central de ciclo combinado.

In [ ]:
if 'google.colab' in str(get_ipython()):
  dataset_path = 'https://raw.githubusercontent.com/cursos-COnCEPT/curso-tensorflow/refs/heads/main/CCP.csv'
else:
  dataset_path = os.getcwd() + '\\CCP.csv'

raw_dataset = pd.read_csv(dataset_path)
dataset = raw_dataset.copy()
dataset.head()

# Split data
train_ratio = .7
val_ratio = .15
df = dataset.sample(frac=train_ratio+val_ratio, random_state=10)
df_test = dataset.drop(df.index)
df_train = df.sample(frac=val_ratio/(val_ratio+train_ratio), random_state=10)
df_valid = df.drop(df_train.index)

# Split features and target
X_train = df_train.drop('PE', axis=1)
X_valid = df_valid.drop('PE', axis=1)
X_test = df_test.drop('PE', axis=1)
y_train = df_train['PE']
y_valid = df_valid['PE']
y_test = df_test['PE']

# Scale to [0, 1]
max_ = X_train.max(axis=0)
min_ = X_train.min(axis=0)
X_train = (X_train - min_) / (max_ - min_)
X_valid = (X_valid - min_) / (max_ - min_)
X_test = (X_test - min_) / (max_ - min_)

## 1- Definición del modelo

Cuando se construye un modelo para *hypertuning*, se definen tanto el espacio de búsqueda de hiperparámetros como la arquitectura del modelo. El modelo que considera rangos de valores para los hiperparámetros en lugar de valores concretos se denomina *hypermodel*.

Aunque hay diferentes alternativas para construir un *hypermodel*, aquí lo haremos utilizando una función, que devolverá un modelo ya compilado.

A continuación, ajusta el número de neuronas de la primera capa, que puede estar comprendido entre 16 y 128, variando en potencias de base 2. Además, escoge el valor óptimo para el learning rate entre las siguientes opciones [0.001, 0.005, 0.0001].

In [ ]:
def model_builder(hp):
    # Define los hiperparámetros
    hp_neurons = # TODO
    hp_learning_rate = # TODO

    # Define el modelo
    model = # TODO

    # Define el optimizador, utilizando Adam como algorimto de optimización y utilizando el valor de learning rate definido en los hiperparámetros
    optimizer = # TODO
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

## 2- Ajuste de hiperparámetros

En Keras hay cuatro métodos diferentes para realizar la búsqueda - `RandomSearch`, `Hyperband`, `BayesianOptimization`, y `Sklearn`. En este ejemplo se utiliza el método `Hyperband`. 

Este algoritmo es rápido y eficiente porque prueba varias combinaciones de opciones (como el número de capas o el *learning rate*) y asigna más recursos (tiempo de entrenamiento) a las opciones que están funcionando mejor, para así encontrar la mejor solución en menos tiempo.

Para instanciar el *tuner*, hay que especificar el hipermodelo, el objetivo a optimizar y el número máximo de épocas a entrenar. 

In [ ]:
# Crea una instancia del tuner
tuner = kt.Hyperband(_______________ , # Indica la función que construye el modelo
                     objective= _____________ , # Indica la métrica a optimizar
                     max_epochs= _______________ , # Indica el número máximo de épocas por cada configuración exploarada
                     overwrite=True)

# Visualiza el resumen del espacio de búsqueda                    
# TODO

Search space summary
Default search space size: 2
neurons (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 2, 'sampling': 'log'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0005, 0.0001], 'ordered': True}


Crea un callback tipo `EarlyStopping` antes de efectuar la búsqueda.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
                        monitor= ______________ , # Métrica para "vigilar"
                        min_delta=______________, # Diferencia mínima a considerar como mejora
                        patience=______________, # Épocas a esperar antes de detener el entrenamiento desde que se deja de mejorar
                        restore_best_weights=True,
                    )

Efectúa la búsqueda, conocida como *hyperparameter tuning* con el comando `tuner.search`. Esta función utiliza los mismos argumentos que el método `model.fit`.

In [ ]:
tuner.search(__________________ , # Completa con el conjunto de inputs de entrenamiento ya normalizadas
             __________________ , # Completa con las outputs del conjunto de datos de entrenamiento
            epochs= ____________________ , # Completa con el número de épocas
            batch_size = ___________________ , # Completa con el tamaño del batch
            validation_data =  __________________ , # Completa con el conjunto de inputs de validación ya normalizadas en forma de tupla (X,y)
            callbacks=[early_stopping],
            verbose='auto')

# Extraer la mejor combinación de hiperparámetros
best_hps= # TODO

print(f"""
El número de neuronas óptimo en la primera capa es {best_hps.get('neurons')} \n y el valor óptimo para el learning rate es {best_hps.get('learning_rate')}.
""")

Trial 12 Complete [00h 00m 02s]
val_loss: 454.1022033691406

Best val_loss So Far: 448.1781311035156
Total elapsed time: 00h 00m 32s

El número de neuronas óptimo en la primera capa es 64 
 y el valor óptimo para el learning rate es 0.001.



## 3- Entrenamiento y evaluación del modelo

Crea una nueva instancia del modelo utilizando la mejor configuración de los hiperparámetros y lleva a cabo el entrenamiento.

In [9]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, 
                    epochs=200,
                    validation_data=(X_valid, y_valid), 
                    batch_size=256,
                    callbacks=[early_stopping],
                    verbose='auto')

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 454.2691 - val_loss: 453.3639
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 452.5218 - val_loss: 450.4492
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 449.2574 - val_loss: 443.7939
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 440.9043 - val_loss: 431.2198
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 426.4490 - val_loss: 410.0909
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 402.5386 - val_loss: 377.9478
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 367.4131 - val_loss: 332.1995
Epoch 8/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 317.9408 - val_loss: 269.8278
Epoch 9/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 250.1926 - val_loss: 188.2901
Epoch 10/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 163.1480 - val_loss: 86.1723
Epoch 11/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68.6296 - val_loss: 51.5347
Epoch 12/200
32/32 ━━━

A continuación, evalúa la calidad del ajuste del modelo.

In [10]:
eval_result = model.evaluate(X_test, y_test)
print("MAE:", eval_result)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.5499 
MAE: 3.59546160697937


## Conclusión
Este ejercicio no ha sido más que una introducción muy breve al tuneado de hiperparámetros. Para aprender más sobre `Keras Tuner`, puedes consultar estos recursos:

* [Keras Tuner en el blog de TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Sitio web del Sintonizador Keras](https://keras-team.github.io/keras-tuner/)